In [1]:
# read in the observation

import pandas as pd
import xarray as xr
import numpy as np


In [2]:
obs = '/home/nils/ramboll/paris/data/donnees_quotidiennes_montsouris.csv'

kw = dict(sep=';', # parse_dates={'time': [0]},
          header='infer', index_col=0, squeeze=True, engine='python') # 

In [3]:
# read into Pandas Series
df = []
df = pd.read_csv(obs, **kw)
# s2 = pd.read_csv(nao_file, **kw)

In [4]:
df.index = pd.to_datetime(df.index)

In [5]:
df['pr'] =  df['pr']/86400
df['tas'] = df['tas'] + 273.15
df['tasmin'] = df['tasmin'] + 273.15
df['tasmax'] = df['tasmax'] + 273.15

In [6]:
df

,pr,tasmin,tasmax,tas,sfcWind
time,,,,,
1872-07-01,NaN,287.85,295.25,291.55,NaN
1872-07-02,NaN,284.35,295.55,289.95,NaN
1872-07-03,NaN,285.25,296.65,290.95,NaN
1872-07-04,NaN,283.75,298.35,291.05,NaN
1872-07-05,NaN,287.55,300.35,293.95,NaN
...,...,...,...,...,...
2020-06-26,0.000007,294.05,304.95,299.50,3.1
2020-06-27,0.000012,291.45,296.55,294.00,4.5
2020-06-28,0.000000,289.35,297.25,293.30,4.0


In [7]:
# open target netCDF 
from netCDF4 import Dataset as ds
from netCDF4 import num2date
from datetime import datetime as dt
import time

In [8]:
tas_file = '/home/nils/ramboll/paris/data/tas_day_montsouris-observation.nc'
# nc_tas = ds(tas_file, 'a')

var_time = nc_tas.variables['time'] 
var_tas = nc_tas.variables['tas']

date_tas = num2date(var_time[:], var_time.units, only_use_cftime_datetimes=False,only_use_python_datetimes=True)
var_tas[:] = np.nan

ids = date_tas.searchsorted(df.index.to_pydatetime())
var_tas[ids] = df['tas'].values

nc_tas.close()

In [12]:
tasmin_file = '/home/nils/ramboll/paris/data/observation/tasmin_day_montsouris-observation.nc'
nc_tasmin = ds(tasmin_file, 'a')

var_time = nc_tasmin.variables['time'] 
var_tasmin = nc_tasmin.variables['tasmin']

date_tasmin = num2date(var_time[:], var_time.units, only_use_cftime_datetimes=False,only_use_python_datetimes=True)
var_tasmin[:] = np.nan

ids = date_tasmin.searchsorted(df.index.to_pydatetime())
var_tasmin[ids] = df['tasmin'].values

nc_tasmin.close()

In [13]:
tasmax_file = '/home/nils/ramboll/paris/data/observation/tasmax_day_montsouris-observation.nc'
nc_tasmax = ds(tasmax_file, 'a')

var_time = nc_tasmax.variables['time'] 
var_tasmax = nc_tasmax.variables['tasmax']

date_tasmax = num2date(var_time[:], var_time.units, only_use_cftime_datetimes=False,only_use_python_datetimes=True)
var_tasmax[:] = np.nan

ids = date_tasmax.searchsorted(df.index.to_pydatetime())
var_tasmax[ids] = df['tasmax'].values

nc_tasmax.close()

In [10]:
pr_file = '/home/nils/ramboll/paris/data/observation/pr_day_montsouris-observation.nc'
nc_pr = ds(pr_file, 'a')

var_pr = nc_pr.variables['pr']
var_pr[:] = np.nan

var_time = nc_pr.variables['time'] 
date_pr = num2date(var_time[:], var_time.units, only_use_cftime_datetimes=False,only_use_python_datetimes=True)

ids = date_pr.searchsorted(df.index.to_pydatetime())
var_pr[ids] = df['pr'].values

nc_pr.close()

In [19]:
### Start the processing
# create filenames 
def drs_filename(resource, skip_timestamp=False, skip_format=False,
                 variable=None, rename_file=False, add_file_path=False):
    
    
    """
    generates filename according to the data reference syntax (DRS)
    based on the metadata in the resource.
    http://cmip-pcmdi.llnl.gov/cmip5/docs/cmip5_data_reference_syntax.pdf
    https://pypi.python.org/pypi/drslib
    :param add_file_path: if add_file_path=True, path to file will be added (default=False)
    :param resource: netcdf file
    :param skip_timestamp: if True then from/to timestamp != added to the filename
                           (default: False)
    :param variable: appropriate variable for filename, if not set (default), variable will
                      be determined. For files with more than one data variable,
                      the variable parameter has to be defined (default: )
                      example: variable='tas'
    :param rename_file: rename the file. (default: False)
    :returns str: DRS filename
    """
    from netCDF4 import Dataset
    from flyingpigeon import nc_utils 
    
    timerange = nc_utils.get_timerange(resource)
    ds = Dataset(resource)
    cmip6_pattern = "{variable}_{frequency}_{model}_{experiment}_{ensemble}_gr_{start}-{end}.nc"  #  tas_day_IPSL-CM6A-LR_historical_r28i1p1f1_gr_18500101-20141231_sub.nc
 
#     if frequency == 'yr':
    start = timerange[0][0:4]
    end = timerange[1][0:4]
    
    filename = cmip6_pattern.format(
                variable=variable,
                frequency = ds.frequency,
                model=ds.source_id,
                experiment=ds.experiment_id,
                ensemble=ds.variant_label, # ds.parent_experiment_rip
                start=start,
                end=end
                )

    return filename
        


In [1]:
from birdy import WPSClient
from os import listdir, path

from shutil import move
from os.path import join
# To display Images from an url
from IPython.core.display import HTML
from IPython.display import Image

In [2]:
finch_url = 'http://localhost:8092'
finch = WPSClient(url=finch_url, progress=True)

In [3]:
p = '/home/nils/Dropbox/Paris_diag_climat - Documents/5_travail/1_phase1/1_1projections/observation/'
tas = join(p, 'tas_day_montsouris-observation.nc')
tasmin = join(p, 'tasmin_day_montsouris-observation.nc')
tasmax = join(p, 'tasmax_day_montsouris-observation.nc')
pr = join(p, 'pr_day_montsouris-observation.nc')

#    

In [4]:
out = finch.wetdays(pr=pr)

In [9]:
finch.tx_mean?

In [5]:
from metalink import download
dl_files = download.get(out.get()[2], path='data/tmp/', segmented=False)

Metalink content-type detected.


In [6]:
# move output files to final destination
var = 'wetdays'

indice_file = '{}_yr_montsouris-observation.nc'.format(var) #    drs_filename(dl_files[0], variable=var)
move(dl_files[0], join(p , 'indices/', indice_file) )

'/home/nils/Dropbox/Paris_diag_climat - Documents/5_travail/1_phase1/1_1projections/observation/indices/wetdays_yr_montsouris-observation.nc'